<img src="https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Creating Data Structures

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

xr.set_options(display_expand_data=False)

rng = np.random.default_rng(seed=0)  # we'll use this later

In the last lecture, we looked at the following example Dataset. In most cases Xarray Datasets are created by reading a file. We'll address this in the next lecture. Here we'll learn how to create Xarray objects from scratch

In [2]:
ds = xr.tutorial.load_dataset("air_temperature")
ds

<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## DataArray

The `DataArray` class is used to attach a name, dimension names, labels, and
attributes to an array.

Our goal will be to recreate the `ds.air` DataArray starting with the underlying numpy data

In [3]:
ds.air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [6]:
array = ds.air.data

We do this using the [DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html) _constructor_.

In [7]:
xr.DataArray(array)

<xarray.DataArray (dim_0: 2920, dim_1: 25, dim_2: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Dimensions without coordinates: dim_0, dim_1, dim_2

This works. Notice that the default dimension names are not so useful: `dim_0`, `dim_1`, `dim_2`


### Dimension Names

We can change this by specifying dimension names in the appropriate order using the `dims` kwarg 

In [8]:
xr.DataArray(array, dims=("time", "lat", "lon"))

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Dimensions without coordinates: time, lat, lon

Much better! But notice we have no entries under "Coordinates".

### Coordinates

While associating names with dimensions (or axes) of an array is quite useful, attaching coordinate labels to DataArrays makes a lot of analysis quite convenient.

First we'll simply add values for `lon` using the `coords` kwarg. For this datasets, longitudes are regularly spaced at 2.5° intervals between 200°E and 330°E.

`coords` takes a dictionary that maps the name of a dimension to one of
- another `DataArray` object
- a tuple of the form `(dims, data, attrs)` where `attrs` is optional. This is
  roughly equivalent to creating a new `DataArray` object with
  `DataArray(dims=dims, data=data, attrs=attrs)`
- a `numpy` array (or anything that can be coerced to one using `numpy.array`).

We'll start with the last one

In [9]:
lon_values = np.arange(200,331,2.5)
xr.DataArray(array, #data
             dims=("time", "lat", "lon"), #dims
             coords = {"lon":lon_values}) # lon coords

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Dimensions without coordinates: time, lat

Assigning a plain numpy array is equivalent to creating a DataArray with those values and the same dimension  name

In [10]:
lon_da = xr.DataArray(lon_values, dims="lon")
da = xr.DataArray(array, dims=("time", "lat", "lon"), coords={"lon": lon_da})
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Dimensions without coordinates: time, lat

We can also assign coordinates after a DataArray has been created.

In [12]:
da.coords["lat"] = np.arange(75, 14.9, -2.5)
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
Dimensions without coordinates: time

### Attributes 

Arbitrary attributes can be assigned using the `.attrs` property

In [14]:
da.attrs["attribute"] = "hello"
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
Dimensions without coordinates: time
Attributes:
    attribute:  hello

or specified in the constructor

In [15]:
da2 = xr.DataArray(
    array, 
    dims=("time", "lat", "lon"), 
    coords={"lon": lon_da}, 
    attrs={"attribute": "hello"}
)
da2

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Dimensions without coordinates: time, lat
Attributes:
    attribute:  hello

### Non-dimension coordinates

Sometimes we want to attach coordinate variables along an existing dimension. Notice that 
1. `itime` is not bolded and 
2. has a name "time" that is different from the dimension name "time"

`itime` is an example of a non-dimension coordinate variable i.e. it is a coordinate variable that does not match a dimension name. Here we demonstrate the "tuple" form of assigninment:  `(dims, data, attrs)`

In [16]:
da.coords["itime"] = ("time", np.arange(2920), {"name": "value"})
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)>
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Attributes:
    attribute:  hello

### Exercises

create a `DataArray` named "height" from random data `rng.random((180, 360)) * 400`

1. with dimensions named "latitude" and "longitude"


In [18]:
xr.DataArray(data= rng.random((180,360))*400,
             dims= ["latitude", "longitude"],
             name="height")

<xarray.DataArray 'height' (latitude: 180, longitude: 360)>
192.4 101.9 45.38 76.42 56.98 3.814 ... 16.73 47.68 199.2 47.88 303.5 121.3
Dimensions without coordinates: latitude, longitude

In [19]:
xr.DataArray(rng.random((180, 360)) * 400, dims=("latitude", "longitude"), name="height")

<xarray.DataArray 'height' (latitude: 180, longitude: 360)>
386.1 179.0 228.3 220.4 128.0 69.01 ... 11.52 5.998 313.2 272.4 333.2 24.13
Dimensions without coordinates: latitude, longitude

2. with dimension coordinates:

- "latitude": -90 to 89 with step size 1
- "longitude": -180 to 179 with step size 1


In [20]:
xr.DataArray(data= rng.random((180,360))*400,
             dims= ["latitude", "longitude"],
             coords={"latitude": np.arange(-90,90,1),
                     "longitude": np.arange(-180,180,1)},
             name="height")

<xarray.DataArray 'height' (latitude: 180, longitude: 360)>
13.55 87.99 159.5 55.55 28.31 266.5 ... 119.4 75.36 211.8 53.95 74.78 241.7
Coordinates:
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -180 -179 -178 -177 -176 ... 176 177 178 179

In [21]:
xr.DataArray(
    rng.random((180, 360)) * 400,
    dims=("latitude", "longitude"),
    coords={"latitude": np.arange(-90, 90, 1), "longitude": np.arange(-180, 180, 1)},
)

<xarray.DataArray (latitude: 180, longitude: 360)>
376.5 224.4 35.41 298.0 35.03 71.69 ... 148.4 119.0 384.3 89.43 160.3 46.98
Coordinates:
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -180 -179 -178 -177 -176 ... 176 177 178 179

3. with metadata for both data and coordinates:

- height: "type": "ellipsoid"
- latitude: "type": "geodetic"
- longitude: "prime_meridian": "greenwich"


In [25]:
xr.DataArray(
    data=rng.random((180, 360)) * 400,
    dims=["latitude", "longitude"],
    coords={
        "latitude": ("latitude", np.arange(-90, 90, 1), {"type": "geodetic"}),
        "longitude": ("longitude", np.arange(-180, 180, 1), {"prime_meridian": "greenwich"}),
    },
    name="height",
    attrs={"type": "ellipsoid"},
)


<xarray.DataArray 'height' (latitude: 180, longitude: 360)>
40.09 36.87 210.0 340.3 57.05 322.1 246.2 ... 371.3 220.8 44.0 392.6 234.0 179.4
Coordinates:
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -180 -179 -178 -177 -176 ... 176 177 178 179
Attributes:
    type:     ellipsoid

In [26]:
xr.DataArray(
    rng.random((180, 360)) * 400,
    dims=("latitude", "longitude"),
    coords={
        "latitude": ("latitude", np.arange(-90, 90, 1), {"type": "geodetic"}),
        "longitude": (
            "longitude",
            np.arange(-180, 180, 1),
            {"prime_meridian": "greenwich"},
        ),
    },
    attrs={"type": "ellipsoid"},
    name="height",
)

<xarray.DataArray 'height' (latitude: 180, longitude: 360)>
27.37 159.0 241.5 378.0 344.3 352.1 ... 54.48 337.0 26.06 248.6 212.8 334.5
Coordinates:
  * latitude   (latitude) int32 -90 -89 -88 -87 -86 -85 ... 84 85 86 87 88 89
  * longitude  (longitude) int32 -180 -179 -178 -177 -176 ... 176 177 178 179
Attributes:
    type:     ellipsoid

## Dataset

`Dataset` objects collect multiple data variables, each with possibly different
dimensions.

The constructor of `Dataset` takes three parameters:

- `data_vars`: dict-like mapping names to values. It has the format described in
  [coordinates](#coordinates) except we need to use either `DataArray` objects
  or the tuple syntax since we have to provide dimension names
- `coords`: same as for `DataArray`
- `attrs`: same as for `Dataset`

Creating an empty Dataset is easy!

In [27]:
xr.Dataset()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

### Data Variables

Let's create a `Dataset` with two data variables: `da` and `da2`

In [30]:
ds = xr.Dataset({"air": da, "air2": da2})
ds

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7

You can directly assign a new data variables

In [31]:
ds["air3"] = da
ds

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air3     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7

### Coordinates

Coordinate variables can be assigned using the `coords` kwarg to `xr.Dataset`. Here we use `date_range` from pandas to create a time vector

In [32]:
xr.Dataset(
    {"air": da, "air2": da2},
    coords={"time": pd.date_range("2013-01-01", "2014-12-31 18:00", freq="6H")},
)

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7

Again we can assign coordinate variables after a Dataset has been created.

In [33]:
ds

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air3     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7

In [34]:
ds.coords["time"] = pd.date_range("2013-01-01", "2014-12-31 18:00", freq="6H")
ds

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air3     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7

### Attributes

In [35]:
xr.Dataset(
    {"air": da, "air2": da2},
    coords={"time": pd.date_range("2013-01-01", "2014-12-31 18:00", freq="6H")},
    attrs={"key0": "value0"},
)

<xarray.Dataset>
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * lat      (lat) float64 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
    itime    (time) int32 0 1 2 3 4 5 6 7 ... 2913 2914 2915 2916 2917 2918 2919
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air2     (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
Attributes:
    key0:     value0

In [36]:
ds.attrs["key"] = "value"

### Exercises

1. create a Dataset with two variables along `latitude` and `longitude`:
   `height` and `gravity_anomaly`


In [39]:
height = rng.random((180, 360)) * 400
gravity_anomaly = rng.random((180, 360)) * 400 - 200

In [40]:
xr.Dataset(
    {
        "height": (("latitude", "longitude"), height),
        "gravity_anomaly": (("latitude", "longitude"), gravity_anomaly),
    }
)


2. add coordinates to `latitude` and `longitude`:

- `latitude`: from -90 to 90 with step size 1
- `longitude`: from -180 to 180 with step size 1


In [ ]:
xr.Dataset(
    {
        "height": (("latitude", "longitude"), height),
        "gravity_anomaly": (("latitude", "longitude"), gravity_anomaly),
    },
    coords={
        "latitude": ("latitude", np.arange(-90, 90, 1)),
        "longitude": ("longitude", np.arange(-180, 180, 1)),
    },
)

3. add metadata to coordinates and variables:

- `latitude`: "type": "geodetic"
- `longitude`: "prime_meridian": "greenwich"
- `height`: "ellipsoid": "wgs84"
- `gravity_anomaly`: "ellipsoid": "grs80"


In [42]:
xr.Dataset(
    {
        "height": (("latitude", "longitude"), height, {"ellipsoid": "wgs84"}),
        "gravity_anomaly": (("latitude", "longitude"), gravity_anomaly, {"ellipsoid": "grs80"}),
    },
    coords={
        "latitude": ("latitude", np.arange(-90, 90, 1), {"type": "geodetic"}),
        "longitude": (
            "longitude",
            np.arange(-180, 180, 1),
            {"prime_meridian": "greenwich"},
        ),
    },
)

<xarray.Dataset>
Dimensions:          (latitude: 180, longitude: 360)
Coordinates:
  * latitude         (latitude) int32 -90 -89 -88 -87 -86 -85 ... 85 86 87 88 89
  * longitude        (longitude) int32 -180 -179 -178 -177 ... 176 177 178 179
Data variables:
    height           (latitude, longitude) float64 24.3 148.5 ... 194.1 164.4
    gravity_anomaly  (latitude, longitude) float64 -167.8 29.35 ... -100.1